# Functions of Arrays

This notebook illustrates how to apply a function to arrays and how to iterate over array elements. Other notebooks focus on how to create matrices and on matrix algebra.

## Load Packages and Extra Functions

In [1]:
using Printf

include("jlFiles/printmat.jl")   #a function for prettier matrix printing

printyellow (generic function with 1 method)

# Elementwise Functions of Arrays: Dot (.)

Let `X` be an array, and `a` and `b` be scalars. Then, `y = fn.(X,a,b)` generates an array `y` where `y[i,j] = fn(X[i,j],a,b)`

(You could achieve the same thing with ```map(xi->fn(xi,a,b),x)```.)

In [2]:
fn(x,a,b) = a/x + b             #x has to be a scalar for this to work

fn (generic function with 1 method)

In [3]:
X = [1 2;
     0 10]

printmat(fn.(X,100,10))         #notice the dot (.)

   110.000    60.000
       Inf    20.000



# Looping over Elements of an Array

There are several ways of looping over all elements in an array. The next cell summarises some of them.

In [4]:
x = [11 12; 21 22]

for z in x                     #when we do not need the index
    println(z)                 #works also with non-traditional arrays
end
println()

y = similar(x)
for i = 1:length(x)            #when we want to use the index i for an output
    y[i] = x[i] + i*100        #works only with traditional arrays (first index is 1)
end
printmat(y)

y = similar(x)
for i in eachindex(x)          #again when we want to use the index i for an output
    y[i] = x[i] + i*100        #more general and would work with non-traditional arrays
end
printmat(y)

11
21
12
22

   111       312    
   221       422    

   111       312    
   221       422    



# Looping over Columns or Rows

Suppose you want to calculate the sum of each row of a matrix. The classical way of doing that is to loop and extract each row as `X[i,:]`

But, there is also a direct way to loop over all rows by using `for r in eachrow(X)`. To also get the row number `i`, we use `for (i,row) in enumerate(eachrow(X))`. For looping over columns, use `eachcol()`.

To change `X` in such a loop, use `row .= new_vector`. Notice the dot (`.`).

In [5]:
X = [1 2;
     0 10]
println("X:")
printmat(X)

m = size(X,1)

z = zeros(Int,m)               #to fill with results, zeros(Int,m) to get Int
for i = 1:m                    #loop over rows
    z[i] = sum(X[i,:])         #or sum(view(X,i,:)) to save memory
end
println("sum of each row:")
printmat(z)

X:
     1         2    
     0        10    

sum of each row:
     3    
    10    



In [6]:
z = zeros(Int,m)
for (i,row) in enumerate(eachrow(X))   #enumerate to get both index and value
    z[i] = sum(row)
end
println("sum of each row:")
printmat(z)

sum of each row:
     3    
    10    



In [7]:
Xb = copy(X) 

for (i,row) in enumerate(eachrow(Xb))   #enumerate to get both index and value
    row .= [i*10+1,i*10+2]              #need .= to change Xb
end

printmat(Xb)

    11        12    
    21        22    



# Functions with Built-in `dims` Argument

May functions have a `dims` argument that allows you to avoid looping, for instance, `sum`.

In [8]:
printmat(sum(X,dims=2))

     3    
    10    



## Performance Tips (extra)

Several functions allow you to also apply an elementwise function to `X` before doing the rest of the calculations, for instance, `any`, `all`, `sum`, `prod`, `maximum`, and `minimum`. This speeds up things since it avoids creating intermediate/temporary arrays.

In [9]:
sum(abs2,X,dims=2)           #same as sum(abs2.(X),dims=2) but faster

2×1 Matrix{Int64}:
   5
 100

# Apply Your Own Function on Each Column: mapslices (extra)

...or each row (or some other dimension)

The `mapslices(fun,x,dims=1)` applies `fun(x[:,i])` to each column of a matrix `x`. This is a convenient alternative to looping over the columns.

The cell below illustrates this by calling a function which calculates the moving average of `x[t]` and `x[t-1]` for each column of a matrix `X`.

In [10]:
function MovingAvg2(x)             #moving average of t and t-1
    T = length(x)
    y = fill(NaN,T)
    for t = 2:T
        y[t] = (x[t] + x[t-1])/2
    end
    return y
end

X = [1:5 101:105]
Y = mapslices(MovingAvg2,X,dims=1)
println("      X (with 2 columns) and Y (MA(1) of X):")
printmat([X Y])

      X (with 2 columns) and Y (MA(1) of X):
     1.000   101.000       NaN       NaN
     2.000   102.000     1.500   101.500
     3.000   103.000     2.500   102.500
     4.000   104.000     3.500   103.500
     5.000   105.000     4.500   104.500



# Working with OffsetArrays (extra)

The next few cells creates an OffsetArray which is an array with non-traditional indexes, see [OffsetArrays.jl](https://github.com/JuliaArrays/OffsetArrays.jl). We then iterate over rows and columns.

In [11]:
using OffsetArrays

A = collect(reshape(1:12,4,3))
println("\nStandard array:")
display(A)

println("\nA matrix with indices -2:1x0:2:")
B = OffsetArray(A,-2:1,0:2)
display(B)


Standard array:


4×3 Matrix{Int64}:
 1  5   9
 2  6  10
 3  7  11
 4  8  12


A matrix with indices -2:1x0:2:


4×3 OffsetArray(::Matrix{Int64}, -2:1, 0:2) with eltype Int64 with indices -2:1×0:2:
 1  5   9
 2  6  10
 3  7  11
 4  8  12

The next cells show two versions of an example of how to loop over rows and columns. The first version requires the correct specification of the row and column indices. 

The second specification is general and would work also on a traditional matrix. For instance, `axes(x,1)[begin+2:end]` returns all the row indices starting at the third row. Alternatively, use`firstindex(x,1)+2:lastindex(x,1)`.

In [12]:
B2 = zero(B)                   
for i = 0:1, j = 1:2    #iterate using known index ranges
  B2[i,j] = B[i-2,j-1]  #iterate over rows 0 and 1, and columns 1 and 2
end
display(B2)

4×3 OffsetArray(::Matrix{Int64}, -2:1, 0:2) with eltype Int64 with indices -2:1×0:2:
 0  0  0
 0  0  0
 0  1  5
 0  2  6

In [13]:
B2 = zero(B)                       ##iterate using general index ranges
for i in axes(B2,1)[begin+2:end], j in axes(B2,2)[begin+1:end]
  B2[i,j] = B[i-2,j-1]
end
display(B2)

4×3 OffsetArray(::Matrix{Int64}, -2:1, 0:2) with eltype Int64 with indices -2:1×0:2:
 0  0  0
 0  0  0
 0  1  5
 0  2  6